# Multi-Class Classification of 16 Personality Types (MBTI)
## Comprehensive ML Techniques Comparison

This notebook compares four traditional machine learning techniques for classifying personality types:
1. **Gradient Boosting (XGBoost)**
2. **Logistic Regression**
3. **Linear Discriminant Analysis (LDA)**
4. **Random Forest**

All models use:
- **Same data splits**: 70/15/15 (Train/Validation/Test)
- **Same random seed**: 42 (for reproducibility)
- **Same dataset**: 16P_eda_cleaned.csv

**Note**: Results may vary slightly between different machines due to hardware-dependent threading behavior.

---
# Step 1: Data Gathering and Exploratory Data Analysis (EDA)

This section performs comprehensive data analysis based on the `data_gathering_eda.py` script to answer:
1. What is the source and size of your dataset?
2. What data quality issues did you encounter?
3. What preprocessing steps did you apply?
4. How did you handle missing values and outliers?
5. What insights did your exploratory analysis reveal?

**Dataset Source:**
> A. Mehta, "60k Responses of 16 Personalities Test (MBT)," Kaggle. 
> Available: https://www.kaggle.com/datasets/anshulmehtakaggl/60k-responses-of-16-personalities-test-mbt/data

In [ ]:
# Import libraries for EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("="*80)
print("DATA GATHERING AND EXPLORATORY DATA ANALYSIS")
print("16 Personalities (MBTI) Dataset")
print("="*80)

## 1.1 Data Source and Size

In [ ]:
# Load the raw dataset
df_raw = pd.read_csv('16P.csv', encoding='cp1252')

print(f"""
Dataset Source:
---------------
Title: 60k Responses of 16 Personalities Test (MBT)
Author: Anshul Mehta
Platform: Kaggle
URL: https://www.kaggle.com/datasets/anshulmehtakaggl/60k-responses-of-16-personalities-test-mbt/data

Dataset Size:
-------------
• Number of Rows (Responses): {df_raw.shape[0]:,}
• Number of Columns (Features): {df_raw.shape[1]}
• Memory Usage: {df_raw.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB

Dataset Structure:
------------------
• 1 Response ID column (identifier)
• 60 Survey Question columns (features)
• 1 Personality Type column (target variable)
""")

# Display sample data
print("First 5 Rows (Sample):")
df_raw.head()

## 1.2 Data Quality Issues

In [ ]:
print("="*80)
print("DATA QUALITY ANALYSIS")
print("="*80)

# 2.1 Check Missing Values
print("\n2.1 Missing Values Analysis")
print("-"*40)
missing_values = df_raw.isnull().sum()
missing_count = missing_values.sum()
if missing_count == 0:
    print("✓ No missing values found in the dataset.")
else:
    print(f"✗ Found {missing_count} missing values:")
    print(missing_values[missing_values > 0])

# 2.2 Check Duplicate Rows
print("\n2.2 Duplicate Rows Analysis")
print("-"*40)
duplicate_count = df_raw.duplicated().sum()
if duplicate_count == 0:
    print("✓ No duplicate rows found in the dataset.")
else:
    print(f"✗ Found {duplicate_count} duplicate rows.")

# 2.3 Check Duplicate Response IDs
print("\n2.3 Duplicate Response IDs")
print("-"*40)
if 'Response Id' in df_raw.columns:
    duplicate_ids = df_raw['Response Id'].duplicated().sum()
    if duplicate_ids == 0:
        print("✓ All Response IDs are unique.")
    else:
        print(f"✗ Found {duplicate_ids} duplicate Response IDs.")

# 2.4 Check Target Variable
print("\n2.4 Target Variable (Personality Types) Validation")
print("-"*40)
valid_types = ['ESTJ', 'ENTJ', 'ESFJ', 'ENFJ', 'ISTJ', 'ISFJ', 'INTJ', 'INFJ',
               'ESTP', 'ESFP', 'ENTP', 'ENFP', 'ISTP', 'ISFP', 'INTP', 'INFP']
unique_personalities = df_raw['Personality'].unique()
invalid_types = [p for p in unique_personalities if p not in valid_types]

print(f"Expected 16 personality types: {len(valid_types)}")
print(f"Found unique values: {len(unique_personalities)}")
if len(invalid_types) == 0:
    print("✓ All personality types are valid MBTI types.")
else:
    print(f"✗ Found invalid personality types: {invalid_types}")

# Summary
print("\n" + "-"*40)
print("DATA QUALITY SUMMARY:")
print("-"*40)
print("✓ The dataset is of HIGH QUALITY with no significant issues!")

## 1.3 Exploratory Data Analysis Visualizations

In [ ]:
# Clean data for visualizations (remove Response Id)
df_eda = df_raw.copy()
if 'Response Id' in df_eda.columns:
    df_eda = df_eda.drop(columns=['Response Id'])

# Figure 1: Personality Type Distribution
print("Figure 1: Personality Type Distribution")
print("-"*40)

personality_counts = df_eda['Personality'].value_counts()

fig, ax = plt.subplots(figsize=(14, 6))
colors = sns.color_palette("husl", 16)
bars = ax.bar(personality_counts.index, personality_counts.values, color=colors)
ax.set_xlabel('Personality Type', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Distribution of 16 Personality Types', fontsize=14, fontweight='bold')
ax.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar, count in zip(bars, personality_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
            f'{count:,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Print statistics
print(f"\nClass Distribution Statistics:")
print(f"Most common type: {personality_counts.idxmax()} ({personality_counts.max():,} samples)")
print(f"Least common type: {personality_counts.idxmin()} ({personality_counts.min():,} samples)")
imbalance_ratio = personality_counts.max() / personality_counts.min()
print(f"Imbalance ratio: {imbalance_ratio:.2f}:1")

In [ ]:
# Figure 2: MBTI Dimensions Pie Charts
print("Figure 2: MBTI Dimensions Distribution")
print("-"*40)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Calculate dimensions
e_count = sum(1 for p in df_eda['Personality'] if p[0] == 'E')
i_count = sum(1 for p in df_eda['Personality'] if p[0] == 'I')
s_count = sum(1 for p in df_eda['Personality'] if p[1] == 'S')
n_count = sum(1 for p in df_eda['Personality'] if p[1] == 'N')
t_count = sum(1 for p in df_eda['Personality'] if p[2] == 'T')
f_count = sum(1 for p in df_eda['Personality'] if p[2] == 'F')
j_count = sum(1 for p in df_eda['Personality'] if p[3] == 'J')
p_count = sum(1 for p in df_eda['Personality'] if p[3] == 'P')

dimensions = [
    (['Extrovert (E)', 'Introvert (I)'], [e_count, i_count], 'Energy: E vs I'),
    (['Sensing (S)', 'Intuition (N)'], [s_count, n_count], 'Information: S vs N'),
    (['Thinking (T)', 'Feeling (F)'], [t_count, f_count], 'Decisions: T vs F'),
    (['Judging (J)', 'Perceiving (P)'], [j_count, p_count], 'Lifestyle: J vs P')
]

colors_pair = ['#FF6B6B', '#4ECDC4']
for ax, (labels, sizes, title) in zip(axes.flat, dimensions):
    wedges, texts, autotexts = ax.pie(sizes, labels=labels, autopct='%1.1f%%', 
                                       colors=colors_pair, startangle=90)
    ax.set_title(title, fontsize=12, fontweight='bold')

plt.suptitle('MBTI Dimensions Distribution', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Print dimension statistics
print(f"\nDimension Distributions:")
print(f"  Extrovert (E) vs Introvert (I): {e_count:,} vs {i_count:,}")
print(f"  Sensing (S) vs Intuition (N):   {s_count:,} vs {n_count:,}")
print(f"  Thinking (T) vs Feeling (F):    {t_count:,} vs {f_count:,}")
print(f"  Judging (J) vs Perceiving (P):  {j_count:,} vs {p_count:,}")

In [ ]:
# Figure 3: Response Value Distribution Heatmap
print("Figure 3: Response Value Distribution Across Questions")
print("-"*40)

feature_cols = [col for col in df_eda.columns if col != 'Personality']

fig, ax = plt.subplots(figsize=(14, 8))

# Get first 15 feature columns for visibility
sample_features = feature_cols[:15]
short_names = [col[:30] + "..." if len(col) > 30 else col for col in sample_features]

# Create distribution matrix
response_values = [-3, -2, -1, 0, 1, 2, 3]
dist_matrix = np.zeros((len(sample_features), len(response_values)))

for i, col in enumerate(sample_features):
    for j, val in enumerate(response_values):
        dist_matrix[i, j] = (df_eda[col] == val).sum()

# Normalize to percentages
dist_matrix_pct = dist_matrix / dist_matrix.sum(axis=1, keepdims=True) * 100

sns.heatmap(dist_matrix_pct, annot=True, fmt='.1f', cmap='YlOrRd',
            xticklabels=response_values, yticklabels=short_names, ax=ax)
ax.set_xlabel('Response Value', fontsize=12)
ax.set_ylabel('Survey Question', fontsize=12)
ax.set_title('Response Distribution Across Questions (%)\n(Showing first 15 questions)', 
             fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Figure 4: Feature Correlation Heatmap
print("Figure 4: Feature Correlation Heatmap")
print("-"*40)

fig, ax = plt.subplots(figsize=(14, 12))

# Select a subset of features for correlation (every 4th question)
sample_features_corr = feature_cols[::4][:15]
short_names_corr = [col[:25] + "..." if len(col) > 25 else col for col in sample_features_corr]

corr_matrix = df_eda[sample_features_corr].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5,
            xticklabels=short_names_corr, yticklabels=short_names_corr, ax=ax)
ax.set_title('Feature Correlation Heatmap\n(Sampled Questions)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Figure 5: Response Distribution Box Plots
print("Figure 5: Response Distribution Box Plots")
print("-"*40)

fig, ax = plt.subplots(figsize=(16, 6))

# Select first 20 features
sample_features_box = feature_cols[:20]
df_melt = df_eda[sample_features_box].melt(var_name='Question', value_name='Response')
df_melt['Question'] = df_melt['Question'].apply(lambda x: x[:20] + "..." if len(x) > 20 else x)

sns.boxplot(data=df_melt, x='Question', y='Response', ax=ax, palette='Set3')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=8)
ax.set_xlabel('Survey Question', fontsize=12)
ax.set_ylabel('Response Value', fontsize=12)
ax.set_title('Response Distribution Across Questions (Box Plots)\n(Showing first 20 questions)', 
             fontsize=14, fontweight='bold')
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 1.4 Key EDA Insights

### Dataset Characteristics
- Large-scale dataset with ~60,000 survey responses
- 60 psychological assessment questions using 7-point Likert scale (-3 to 3)
- Covers all 16 MBTI personality types

### Data Quality
- ✓ Excellent data quality - no missing values detected
- ✓ No duplicate entries or invalid personality types
- ✓ All responses within expected value range (-3 to 3)

### Class Distribution Insights
- Some class imbalance exists between personality types (natural distribution)
- Stratified sampling recommended during model training

### Preprocessing Applied
1. Removed 'Response Id' column (identifier not useful for prediction)
2. Verified all feature columns are numeric
3. No missing value imputation needed

### Conclusions
The dataset is well-prepared for multi-class classification using machine learning techniques.

---
## Setup and Data Loading

### ⚠️ IMPORTANT: Run this cell first in Google Colab, then restart runtime!
Skip this cell if running locally.

In [ ]:
# ========================================
# REPRODUCIBILITY SETUP (Run in Colab only)
# ========================================
# Install exact library versions matching local environment
# After running this cell, restart the runtime: Runtime -> Restart runtime

!pip install numpy==2.3.3 pandas==2.3.2 scikit-learn==1.7.2 xgboost==3.1.3 -q

print("✓ Libraries installed. Please restart runtime now!")
print("Go to: Runtime -> Restart runtime, then run all cells below.")

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    top_k_accuracy_score
)
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Print library versions for verification
print(f"numpy: {np.__version__}")
print(f"pandas: {pd.__version__}")
import sklearn; print(f"scikit-learn: {sklearn.__version__}")
import xgboost; print(f"xgboost: {xgboost.__version__}")
print("\n✓ All libraries imported successfully")

In [ ]:
# Use the cleaned data from EDA section (df_eda already prepared above)
# Rename to df for consistency with the rest of the notebook
df = df_eda.copy()

print(f"Dataset loaded from EDA: {len(df):,} rows and {len(df.columns)} columns")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Separate features and target
X = df.drop(columns=['Personality'])
y = df['Personality']

feature_columns = X.columns.tolist()
print(f"Features: {len(feature_columns)} survey questions")
print(f"Target: 16 MBTI personality types")
print(f"\nClass distribution:")
print(y.value_counts().sort_index())

---
## Data Preprocessing

In [ ]:
# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
class_names = label_encoder.classes_
print(f"Encoded {len(class_names)} classes: {', '.join(class_names)}")

In [ ]:
# Split data (70% Train, 15% Validation, 15% Test)
TEST_SIZE = 0.15      # 15% for test
VAL_SIZE = 0.176      # 15% of remaining 85% ≈ 15% of total

# First split: 85% train+val, 15% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_encoded, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE, 
    stratify=y_encoded
)

# Second split: 70% train, 15% val (of total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, 
    test_size=VAL_SIZE, 
    random_state=RANDOM_STATE, 
    stratify=y_temp
)

print(f"Training set:   {len(X_train):,} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"Validation set: {len(X_val):,} samples ({len(X_val)/len(df)*100:.1f}%)")
print(f"Test set:       {len(X_test):,} samples ({len(X_test)/len(df)*100:.1f}%)")

In [ ]:
# Feature scaling (for Logistic Regression and LDA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✓ Features standardized to zero mean and unit variance")

---
## Model 1: Gradient Boosting (XGBoost)

In [ ]:
print("=" * 80)
print("TRAINING GRADIENT BOOSTING (XGBoost) CLASSIFIER")
print("=" * 80)

# Configure XGBoost (matching local xgboost_classifier.py)
xgb_params = {
    'n_estimators': 500,
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'multi:softprob',
    'num_class': 16,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbosity': 0
}

print("\nHyperparameters:")
for key, value in xgb_params.items():
    if key not in ['verbosity', 'n_jobs']:
        print(f"  {key}: {value}")

# Train model
xgb_model = XGBClassifier(**xgb_params, early_stopping_rounds=15)
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

print(f"\n✓ Training complete! Best iteration: {xgb_model.best_iteration}")

In [ ]:
# Make predictions
xgb_train_pred = xgb_model.predict(X_train)
xgb_val_pred = xgb_model.predict(X_val)
xgb_test_pred = xgb_model.predict(X_test)
xgb_test_proba = xgb_model.predict_proba(X_test)

# Calculate metrics
xgb_train_acc = accuracy_score(y_train, xgb_train_pred)
xgb_val_acc = accuracy_score(y_val, xgb_val_pred)
xgb_test_acc = accuracy_score(y_test, xgb_test_pred)
xgb_f1 = f1_score(y_test, xgb_test_pred, average='macro')
xgb_precision = precision_score(y_test, xgb_test_pred, average='macro')
xgb_recall = recall_score(y_test, xgb_test_pred, average='macro')
xgb_top3 = top_k_accuracy_score(y_test, xgb_test_proba, k=3)

print("GRADIENT BOOSTING RESULTS")
print("-" * 50)
print(f"Training Accuracy:   {xgb_train_acc:.4f} ({xgb_train_acc*100:.2f}%)")
print(f"Validation Accuracy: {xgb_val_acc:.4f} ({xgb_val_acc*100:.2f}%)")
print(f"Test Accuracy:       {xgb_test_acc:.4f} ({xgb_test_acc*100:.2f}%)")
print(f"\nTop-3 Accuracy:      {xgb_top3:.4f} ({xgb_top3*100:.2f}%)")
print(f"Macro Precision:     {xgb_precision:.4f}")
print(f"Macro Recall:        {xgb_recall:.4f}")
print(f"Macro F1-Score:      {xgb_f1:.4f}")

xgb_cm = confusion_matrix(y_test, xgb_test_pred)

---
## Model 2: Logistic Regression

In [ ]:
print("=" * 80)
print("TRAINING LOGISTIC REGRESSION CLASSIFIER")
print("=" * 80)

# Configure Logistic Regression
lr_params = {
    'multi_class': 'multinomial',
    'solver': 'lbfgs',
    'max_iter': 1000,
    'C': 1.0,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': 0
}

print("\nHyperparameters:")
for key, value in lr_params.items():
    if key not in ['verbose', 'n_jobs']:
        print(f"  {key}: {value}")

# Train model (uses scaled data)
lr_model = LogisticRegression(**lr_params)
lr_model.fit(X_train_scaled, y_train)

print(f"\n✓ Training complete! Iterations: {lr_model.n_iter_[0]}")

In [ ]:
# Make predictions
lr_train_pred = lr_model.predict(X_train_scaled)
lr_val_pred = lr_model.predict(X_val_scaled)
lr_test_pred = lr_model.predict(X_test_scaled)
lr_test_proba = lr_model.predict_proba(X_test_scaled)

# Calculate metrics
lr_train_acc = accuracy_score(y_train, lr_train_pred)
lr_val_acc = accuracy_score(y_val, lr_val_pred)
lr_test_acc = accuracy_score(y_test, lr_test_pred)
lr_f1 = f1_score(y_test, lr_test_pred, average='macro')
lr_precision = precision_score(y_test, lr_test_pred, average='macro')
lr_recall = recall_score(y_test, lr_test_pred, average='macro')
lr_top3 = top_k_accuracy_score(y_test, lr_test_proba, k=3)

print("LOGISTIC REGRESSION RESULTS")
print("-" * 50)
print(f"Training Accuracy:   {lr_train_acc:.4f} ({lr_train_acc*100:.2f}%)")
print(f"Validation Accuracy: {lr_val_acc:.4f} ({lr_val_acc*100:.2f}%)")
print(f"Test Accuracy:       {lr_test_acc:.4f} ({lr_test_acc*100:.2f}%)")
print(f"\nTop-3 Accuracy:      {lr_top3:.4f} ({lr_top3*100:.2f}%)")
print(f"Macro Precision:     {lr_precision:.4f}")
print(f"Macro Recall:        {lr_recall:.4f}")
print(f"Macro F1-Score:      {lr_f1:.4f}")

lr_cm = confusion_matrix(y_test, lr_test_pred)

---
## Model 3: Linear Discriminant Analysis (LDA)

In [ ]:
print("=" * 80)
print("TRAINING LINEAR DISCRIMINANT ANALYSIS (LDA) CLASSIFIER")
print("=" * 80)

# Configure LDA
lda_params = {
    'solver': 'svd',
    'n_components': None,
    'store_covariance': False,
    'tol': 1e-4
}

print("\nHyperparameters:")
for key, value in lda_params.items():
    print(f"  {key}: {value}")

# Train model (uses scaled data)
lda_model = LinearDiscriminantAnalysis(**lda_params)
lda_model.fit(X_train_scaled, y_train)

n_components = lda_model.scalings_.shape[1]
print(f"\n✓ Training complete! Discriminant components: {n_components}")
print(f"Explained variance ratio (first 5): {lda_model.explained_variance_ratio_[:5].round(4).tolist()}")

In [ ]:
# Make predictions
lda_train_pred = lda_model.predict(X_train_scaled)
lda_val_pred = lda_model.predict(X_val_scaled)
lda_test_pred = lda_model.predict(X_test_scaled)
lda_test_proba = lda_model.predict_proba(X_test_scaled)

# Calculate metrics
lda_train_acc = accuracy_score(y_train, lda_train_pred)
lda_val_acc = accuracy_score(y_val, lda_val_pred)
lda_test_acc = accuracy_score(y_test, lda_test_pred)
lda_f1 = f1_score(y_test, lda_test_pred, average='macro')
lda_precision = precision_score(y_test, lda_test_pred, average='macro')
lda_recall = recall_score(y_test, lda_test_pred, average='macro')
lda_top3 = top_k_accuracy_score(y_test, lda_test_proba, k=3)

print("LINEAR DISCRIMINANT ANALYSIS RESULTS")
print("-" * 50)
print(f"Training Accuracy:   {lda_train_acc:.4f} ({lda_train_acc*100:.2f}%)")
print(f"Validation Accuracy: {lda_val_acc:.4f} ({lda_val_acc*100:.2f}%)")
print(f"Test Accuracy:       {lda_test_acc:.4f} ({lda_test_acc*100:.2f}%)")
print(f"\nTop-3 Accuracy:      {lda_top3:.4f} ({lda_top3*100:.2f}%)")
print(f"Macro Precision:     {lda_precision:.4f}")
print(f"Macro Recall:        {lda_recall:.4f}")
print(f"Macro F1-Score:      {lda_f1:.4f}")

lda_cm = confusion_matrix(y_test, lda_test_pred)

---
## Model 4: Random Forest

In [ ]:
print("=" * 80)
print("TRAINING RANDOM FOREST CLASSIFIER")
print("=" * 80)

# Configure Random Forest
rf_params = {
    'n_estimators': 100,
    'max_depth': 20,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': 0
}

print("\nHyperparameters:")
for key, value in rf_params.items():
    if key not in ['verbose', 'n_jobs']:
        print(f"  {key}: {value}")

# Train model
rf_model = RandomForestClassifier(**rf_params)
rf_model.fit(X_train, y_train)

print(f"\n✓ Training complete! Number of trees: {len(rf_model.estimators_)}")

In [ ]:
# Make predictions
rf_train_pred = rf_model.predict(X_train)
rf_val_pred = rf_model.predict(X_val)
rf_test_pred = rf_model.predict(X_test)
rf_test_proba = rf_model.predict_proba(X_test)

# Calculate metrics
rf_train_acc = accuracy_score(y_train, rf_train_pred)
rf_val_acc = accuracy_score(y_val, rf_val_pred)
rf_test_acc = accuracy_score(y_test, rf_test_pred)
rf_f1 = f1_score(y_test, rf_test_pred, average='macro')
rf_precision = precision_score(y_test, rf_test_pred, average='macro')
rf_recall = recall_score(y_test, rf_test_pred, average='macro')
rf_top3 = top_k_accuracy_score(y_test, rf_test_proba, k=3)

print("RANDOM FOREST RESULTS")
print("-" * 50)
print(f"Training Accuracy:   {rf_train_acc:.4f} ({rf_train_acc*100:.2f}%)")
print(f"Validation Accuracy: {rf_val_acc:.4f} ({rf_val_acc*100:.2f}%)")
print(f"Test Accuracy:       {rf_test_acc:.4f} ({rf_test_acc*100:.2f}%)")
print(f"\nTop-3 Accuracy:      {rf_top3:.4f} ({rf_top3*100:.2f}%)")
print(f"Macro Precision:     {rf_precision:.4f}")
print(f"Macro Recall:        {rf_recall:.4f}")
print(f"Macro F1-Score:      {rf_f1:.4f}")

rf_cm = confusion_matrix(y_test, rf_test_pred)

---
# 📊 MODEL COMPARISONS

## 1. Overall Performance Comparison

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': ['Gradient Boosting', 'Logistic Regression', 'LDA', 'Random Forest'],
    'Train Accuracy': [xgb_train_acc, lr_train_acc, lda_train_acc, rf_train_acc],
    'Val Accuracy': [xgb_val_acc, lr_val_acc, lda_val_acc, rf_val_acc],
    'Test Accuracy': [xgb_test_acc, lr_test_acc, lda_test_acc, rf_test_acc],
    'Top-3 Accuracy': [xgb_top3, lr_top3, lda_top3, rf_top3],
    'Macro Precision': [xgb_precision, lr_precision, lda_precision, rf_precision],
    'Macro Recall': [xgb_recall, lr_recall, lda_recall, rf_recall],
    'Macro F1-Score': [xgb_f1, lr_f1, lda_f1, rf_f1]
})

# Add overfitting gap
comparison_df['Overfit Gap'] = comparison_df['Train Accuracy'] - comparison_df['Test Accuracy']

# Display with formatting
print("=" * 120)
print("COMPREHENSIVE MODEL COMPARISON TABLE")
print("=" * 120)
print(comparison_df.to_string(index=False))
print("=" * 120)

# Find best performers
best_test_acc = comparison_df.loc[comparison_df['Test Accuracy'].idxmax(), 'Model']
best_f1 = comparison_df.loc[comparison_df['Macro F1-Score'].idxmax(), 'Model']
best_top3 = comparison_df.loc[comparison_df['Top-3 Accuracy'].idxmax(), 'Model']
least_overfit = comparison_df.loc[comparison_df['Overfit Gap'].idxmin(), 'Model']

print(f"\n🏆 BEST PERFORMERS:")
print(f"  • Highest Test Accuracy: {best_test_acc}")
print(f"  • Highest Macro F1-Score: {best_f1}")
print(f"  • Highest Top-3 Accuracy: {best_top3}")
print(f"  • Least Overfitting: {least_overfit}")

comparison_df

## 2. Visual Comparison: Test Accuracy

In [ ]:
# Bar chart comparing test accuracies
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#1f77b4', '#2ca02c', '#9467bd', '#ff7f0e']
bars = ax.bar(comparison_df['Model'], comparison_df['Test Accuracy'], color=colors, alpha=0.8)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}\n({height*100:.2f}%)',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_ylabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Model Comparison: Test Set Accuracy', fontsize=14, fontweight='bold')
ax.set_ylim(0, 1)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

## 3. Visual Comparison: All Metrics

In [ ]:
# Multi-metric comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comprehensive Model Performance Comparison', fontsize=16, fontweight='bold')

metrics = [
    ('Test Accuracy', 'Test Accuracy'),
    ('Macro F1-Score', 'Macro F1-Score'),
    ('Top-3 Accuracy', 'Top-3 Accuracy'),
    ('Overfit Gap', 'Overfitting Gap (lower is better)')
]

for idx, (metric, title) in enumerate(metrics):
    row, col = idx // 2, idx % 2
    ax = axes[row, col]
    
    # Use different color for overfitting (lower is better)
    if metric == 'Overfit Gap':
        color_palette = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']  # Red to blue
    else:
        color_palette = colors
    
    bars = ax.bar(comparison_df['Model'], comparison_df[metric], color=color_palette, alpha=0.8)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    ax.set_ylabel(metric, fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    ax.tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.show()

## 4. Train vs Test Accuracy (Overfitting Analysis)

In [ ]:
# Overfitting visualization
fig, ax = plt.subplots(figsize=(12, 7))

x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['Train Accuracy'], width, 
               label='Train Accuracy', color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, comparison_df['Test Accuracy'], width,
               label='Test Accuracy', color='coral', alpha=0.8)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Train vs Test Accuracy - Overfitting Analysis', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Model'], rotation=15)
ax.legend(fontsize=11)
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 OVERFITTING ANALYSIS:")
for idx, row in comparison_df.iterrows():
    gap = row['Overfit Gap']
    status = "✓ Good generalization" if gap < 0.05 else "⚠ Potential overfitting"
    print(f"  {row['Model']:20s}: Gap = {gap:.4f} ({gap*100:.2f}%) - {status}")

## 5. Confusion Matrix Comparison

In [ ]:
# Plot all confusion matrices side by side
fig, axes = plt.subplots(2, 2, figsize=(20, 18))
fig.suptitle('Confusion Matrix Comparison - All Models', fontsize=16, fontweight='bold', y=0.995)

cms = [xgb_cm, lr_cm, lda_cm, rf_cm]
model_names = ['Gradient Boosting (XGBoost)', 'Logistic Regression', 'LDA', 'Random Forest']
cmaps = ['Blues', 'Greens', 'Purples', 'Oranges']

for idx, (cm, name, cmap) in enumerate(zip(cms, model_names, cmaps)):
    row, col = idx // 2, idx % 2
    ax = axes[row, col]
    
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, ax=ax,
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    
    ax.set_title(f'{name}\nTest Accuracy: {comparison_df.iloc[idx]["Test Accuracy"]:.4f}', 
                 fontsize=12, fontweight='bold')
    ax.set_xlabel('Predicted', fontsize=10)
    ax.set_ylabel('Actual', fontsize=10)

plt.tight_layout()
plt.show()

## 6. Per-Class Accuracy Comparison

In [ ]:
# Calculate per-class accuracy for all models
xgb_per_class = xgb_cm.diagonal() / xgb_cm.sum(axis=1)
lr_per_class = lr_cm.diagonal() / lr_cm.sum(axis=1)
lda_per_class = lda_cm.diagonal() / lda_cm.sum(axis=1)
rf_per_class = rf_cm.diagonal() / rf_cm.sum(axis=1)

# Create comparison plot
fig, ax = plt.subplots(figsize=(16, 8))

x = np.arange(len(class_names))
width = 0.2

ax.bar(x - 1.5*width, xgb_per_class, width, label='Gradient Boosting', color='#1f77b4', alpha=0.8)
ax.bar(x - 0.5*width, lr_per_class, width, label='Logistic Regression', color='#2ca02c', alpha=0.8)
ax.bar(x + 0.5*width, lda_per_class, width, label='LDA', color='#9467bd', alpha=0.8)
ax.bar(x + 1.5*width, rf_per_class, width, label='Random Forest', color='#ff7f0e', alpha=0.8)

ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_xlabel('Personality Type', fontsize=12, fontweight='bold')
ax.set_title('Per-Class Accuracy Comparison Across All Models', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(class_names, rotation=45)
ax.legend(fontsize=11, loc='lower right')
ax.set_ylim(0, 1)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Feature Importance Comparison (Top 15 Features)

In [ ]:
# Get feature importance from each model
xgb_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

lr_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': np.mean(np.abs(lr_model.coef_), axis=0)
}).sort_values('Importance', ascending=False)

lda_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': np.mean(np.abs(lda_model.coef_), axis=0)
}).sort_values('Importance', ascending=False)

rf_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Display top 15 features for each model
fig, axes = plt.subplots(2, 2, figsize=(18, 16))
fig.suptitle('Top 15 Most Important Features - All Models', fontsize=16, fontweight='bold')

importances = [xgb_importance, lr_importance, lda_importance, rf_importance]
titles = ['Gradient Boosting', 'Logistic Regression', 'LDA', 'Random Forest']
colors_imp = ['steelblue', 'forestgreen', 'mediumpurple', 'coral']

for idx, (importance_df, title, color) in enumerate(zip(importances, titles, colors_imp)):
    row, col = idx // 2, idx % 2
    ax = axes[row, col]
    
    top_15 = importance_df.head(15)
    short_names = [f[:35] + "..." if len(f) > 35 else f for f in top_15['Feature']]
    
    ax.barh(range(len(top_15)), top_15['Importance'].values, color=color, alpha=0.8)
    ax.set_yticks(range(len(top_15)))
    ax.set_yticklabels(short_names, fontsize=9)
    ax.invert_yaxis()
    ax.set_xlabel('Importance Score', fontsize=10, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Statistical Summary

In [ ]:
print("=" * 100)
print("STATISTICAL SUMMARY ACROSS ALL MODELS")
print("=" * 100)

summary_stats = comparison_df[['Test Accuracy', 'Macro F1-Score', 'Top-3 Accuracy', 'Overfit Gap']].describe()
print(summary_stats)

print("\n" + "=" * 100)
print("MODEL RANKINGS")
print("=" * 100)

# Rank models by different metrics
metrics_to_rank = ['Test Accuracy', 'Macro F1-Score', 'Top-3 Accuracy']
for metric in metrics_to_rank:
    sorted_df = comparison_df.sort_values(metric, ascending=False)
    print(f"\n{metric}:")
    for rank, (idx, row) in enumerate(sorted_df.iterrows(), 1):
        print(f"  {rank}. {row['Model']:20s} - {row[metric]:.4f}")

# Overfitting ranking (lower is better)
sorted_df = comparison_df.sort_values('Overfit Gap', ascending=True)
print(f"\nLeast Overfitting (Train-Test Gap):")
for rank, (idx, row) in enumerate(sorted_df.iterrows(), 1):
    print(f"  {rank}. {row['Model']:20s} - {row['Overfit Gap']:.4f}")

---
## 🎯 Final Recommendations

In [ ]:
print("=" * 100)
print("FINAL RECOMMENDATIONS & KEY INSIGHTS")
print("=" * 100)

# Find best overall model
best_idx = comparison_df['Test Accuracy'].idxmax()
best_model = comparison_df.iloc[best_idx]

print(f"\n🏆 BEST OVERALL MODEL: {best_model['Model']}")
print(f"   Test Accuracy: {best_model['Test Accuracy']:.4f} ({best_model['Test Accuracy']*100:.2f}%)")
print(f"   Macro F1-Score: {best_model['Macro F1-Score']:.4f}")
print(f"   Top-3 Accuracy: {best_model['Top-3 Accuracy']:.4f} ({best_model['Top-3 Accuracy']*100:.2f}%)")
print(f"   Overfitting Gap: {best_model['Overfit Gap']:.4f} ({best_model['Overfit Gap']*100:.2f}%)")

print("\n📊 KEY INSIGHTS:")
print("\n1. ACCURACY PERFORMANCE:")
print(f"   • Highest test accuracy: {comparison_df['Test Accuracy'].max():.4f}")
print(f"   • Lowest test accuracy: {comparison_df['Test Accuracy'].min():.4f}")
print(f"   • Range: {(comparison_df['Test Accuracy'].max() - comparison_df['Test Accuracy'].min()):.4f}")

print("\n2. GENERALIZATION:")
overfitting_models = comparison_df[comparison_df['Overfit Gap'] > 0.05]
if len(overfitting_models) > 0:
    print(f"   • {len(overfitting_models)} model(s) show potential overfitting:")
    for idx, row in overfitting_models.iterrows():
        print(f"     - {row['Model']}: Gap = {row['Overfit Gap']:.4f}")
else:
    print("   • All models generalize well (overfit gap < 5%)")

print("\n3. TOP-3 ACCURACY:")
print(f"   • Best top-3 accuracy: {comparison_df['Top-3 Accuracy'].max():.4f} ({best_top3})")
print(f"   • All models achieve >70% top-3 accuracy" if comparison_df['Top-3 Accuracy'].min() > 0.7 else "")

print("\n4. RECOMMENDATIONS:")
print(f"   • For HIGHEST ACCURACY: Use {best_test_acc}")
print(f"   • For BEST BALANCED PERFORMANCE: Use {best_f1}")
print(f"   • For LEAST OVERFITTING: Use {least_overfit}")

print("\n" + "=" * 100)
print("ANALYSIS COMPLETE")
print("=" * 100)

---
## 📁 Export Results

In [ ]:
# Save comparison results to CSV
comparison_df.to_csv('model_comparison_results.csv', index=False)
print("✓ Saved: model_comparison_results.csv")

# Save per-class accuracies
per_class_df = pd.DataFrame({
    'Personality Type': class_names,
    'Gradient Boosting': xgb_per_class,
    'Logistic Regression': lr_per_class,
    'LDA': lda_per_class,
    'Random Forest': rf_per_class
})
per_class_df.to_csv('per_class_accuracy_comparison.csv', index=False)
print("✓ Saved: per_class_accuracy_comparison.csv")

print("\n✅ All results exported successfully!")